# Setup for the sample
**Pre-requisites**:
- an active MLflow tracking server managed by Amazon SageMaker
- the IAM role associated with the notebook must have sufficient permissions to write to the MLflow tracking server
- Access via Amazon Bedrock to Anthropic Clause v3 Haiku and Sonnet

In [ ]:
import json

import sagemaker
from ipywidgets import interactive

In [ ]:
sagemaker_session = sagemaker.Session()
sagemaker_client = sagemaker_session.sagemaker_client

In [ ]:
def get_studio_username() -> str | None:
    """Attempt tog get user name from Studio environment"""
    try:
        from sagemaker.session import NOTEBOOK_METADATA_FILE

        with open(NOTEBOOK_METADATA_FILE, "rb") as f:
            nb_metadata = json.load(f)
        if user_name := nb_metadata.get("UserProfileName"):
            return user_name
        return sagemaker_client.describe_space(
            DomainId=nb_metadata.get("DomainId"),
            SpaceName=nb_metadata.get("SpaceName"),
        )["OwnershipSettings"]["OwnerUserProfileName"]
    except Exception as e:
        print(e)
        return

In [ ]:
tracking_servers_list = {
    k.get("TrackingServerName"): k.get("TrackingServerArn")
    for k in sagemaker_client.list_mlflow_tracking_servers(
        # TrackingServerStatus='Started'
    )["TrackingServerSummaries"]
}

select_tracking_server = interactive(lambda x: x, x=tracking_servers_list)
select_tracking_server

## Write .env file

In [ ]:
with open(".env", "w") as f:
    f.write(f"MLFLOW_TRACKING_USERNAME={get_studio_username()}\n")
    f.write(f"MLFLOW_TRACKING_URI={select_tracking_server.result}\n")

In [ ]:
!cat .env

## Install dependencies

We install `fmeval-mlflow` directly from the repository.

In [ ]:
%pip install ..

And then install additional dependencies necessary for running the examples notebooks.

In [ ]:
%pip install -r requirements.txt -U